In [1]:
import os
import sys
#sys.path.insert(0, os.path.dirname(os.getcwd()))
import pickle

import numpy as np
import torch
#import torch.cuda
import timeit
import pandas as pd
#import argparse
import itertools
from load_data import load_data # segment_SHAP
from evaluation.metrics.AUC_difference import AUC_difference
from predictor_utils import load_predictor, predict_proba
from pickle import dump



In [2]:
# PATHS
datasets_folder_path = None
attributions_folder_path = None
trained_models_folder_path = None
save_results_folder_path = None

In [3]:
# settings
dataset_names = ['MP']    #[sys.argv[1]]
predictor_names = ['randomForest']    #[sys.argv[2]] ["randomForest", 'miniRocket', 'resNet', "QUANT"]
segmentation_names = ["clasp","greedygaussian", "equal", "infogain","nnsegment"] # ["clasp","greedygaussian", "equal", "infogain","nnsegment"] 
background_names =  ["average", "zero","sampling"] #["average", "zero", "sampling"]
normalization_names = ["default", "normalized"]

metric_names = ["AUC_difference"]

demo_mode = True
# demo
if demo_mode:
    dataset_names = ['UWAVE']
    predictor_names = ['resNet', 'randomForest'] #['randomForest', 'resNet', 'miniRocket']
    segmentation_names = ['clasp']
    background_names = ["average", 'zero'] #,'sampling']
    normalization_names = ["default", "normalized"]

# optional
# # get infos about which explanations are evaluated
# datasets = list( explanations['attributions'].keys() )
# segmentations = list( explanations['attributions'][datasets[0]].keys() )
# predictors = list( explanations['attributions'][datasets[0]][segmentations[0]].keys() )
# backgrounds = list( explanations['attributions'][datasets[0]][segmentations[0]][predictors[0]].keys() )
# result_types = ['default','normalized']

In [9]:
cwd = os.getcwd()
if datasets_folder_path is None:
    datasets_folder_path = "datasets" #os.path.join(cwd, "datasets")
if attributions_folder_path is None:
    attributions_folder_path = "attributions"
if trained_models_folder_path is None:
    trained_models_folder_path = "trained_models"
if save_results_folder_path is None:
    save_results_folder_path = os.path.join("evaluation", "results")

# device
device = "cuda" if torch.cuda.is_available() else "cpu"

starttime = timeit.default_timer()

for dataset_name in dataset_names:
    print("Dataset: ", dataset_name)
    # loading dataset
    X_train, X_test, y_train, y_test, enc = load_data(subset='all', dataset_name=dataset_name, path=datasets_folder_path)
    if demo_mode:
        X_test, y_test = X_test[:2], y_test[:2]


    for predictor_name in predictor_names:
        print("Predictor: ", predictor_name)

        # load classifier
        predictor = load_predictor(path=trained_models_folder_path, predictor_name=predictor_name, dataset_name=dataset_name, device=device) # torch.device(device)

        # load explanations
        attribution_filename = "_".join(("all_results", dataset_name, predictor_name)) + ".npy"
        explanations = np.load(os.path.join(attributions_folder_path, attribution_filename), allow_pickle=True).item() # FileNotFoundError
        label_mapping = explanations['label_mapping'][dataset_name]

        data_list = []

        for key in itertools.product(segmentation_names, background_names, normalization_names, metric_names):
            segmentation_name, background_name, normalization_name, metric_name = key
            print("assessing", key)

            # load model and explanations to access
            try:
                attributions = explanations['attributions'][dataset_name][segmentation_name][predictor_name][background_name][normalization_name]
            except KeyError as error:
                print('Warning: attributions is missing keys, skipping to next ' + repr(error))
                continue
            try:
                y_test_pred = explanations['y_test_pred'][dataset_name][predictor_name]
            except KeyError:
                y_test_pred = explanations['y_test_pred'][dataset_name][segmentation_name][predictor_name]

            if metric_name=="AUC_difference":
                results = AUC_difference(predictor, X_train, X_test, y_train, attributions, y_test_pred, label_mapping, n_steps=18)
            else:
                raise ValueError(f"metric name {metric_name} is not supported")
            for result_tuple in results:
                data_list.append((dataset_name, segmentation_name, predictor_name, background_name, normalization_name, metric_name) + result_tuple)

            print("elapsed time", (timeit.default_timer() - starttime))

        # save
        column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Metric', 'Perturb', "Result"]
        df = pd.DataFrame(data=data_list, columns = column_names)
        file_name = "_".join(("evaluation", predictor_name, dataset_name))
        result_path = os.path.join(save_results_folder_path, file_name)
        if not demo_mode:
            df.to_csv(result_path)
            # with open( "_".join( (dataset_name,classifier_name)) ,"wb") as f:
            #     pickle.dump(results_dict, f)



Dataset:  UWAVE
Predictor:  resNet
assessing ('clasp', 'average', 'default', 'AUC_difference')
(428, 8)
elapsed time 0.9670077000046149
assessing ('clasp', 'average', 'normalized', 'AUC_difference')
(428, 8)
elapsed time 1.7912940999958664
assessing ('clasp', 'zero', 'default', 'AUC_difference')
(428, 8)
elapsed time 2.568037200020626
assessing ('clasp', 'zero', 'normalized', 'AUC_difference')
(428, 8)
elapsed time 3.3488725000061095
Predictor:  randomForest
assessing ('clasp', 'average', 'default', 'AUC_difference')
(428, 8)
elapsed time 4.052089700009674
assessing ('clasp', 'average', 'normalized', 'AUC_difference')
(428, 8)
elapsed time 4.651207800023258
assessing ('clasp', 'zero', 'default', 'AUC_difference')
(428, 8)
elapsed time 5.204801800020505
assessing ('clasp', 'zero', 'normalized', 'AUC_difference')
(428, 8)
elapsed time 5.76889609999489


In [5]:
# column_names = ['Dataset', 'Segmentation', 'ML model', 'Background', 'Normalization', 'Metric', 'Perturb', "Result"]
# df = pd.DataFrame(data=data_list, columns = column_names)

In [6]:
# df

(428, 8)